In [2]:
import multiprocessing as mp

# Foundational SSM core imports
from foundational_ssm.loaders import get_brainset_data_loader
from foundational_ssm.constants import DATA_ROOT
from foundational_ssm.samplers import TrialSampler
import os 
import equinox as eqx


data_root = '../' + DATA_ROOT # change to the folder holding the brainsets
config_dir = '../configs/dataset' # change
dataset_args = {
    'keep_files_open': False,
    'lazy': True,
    'split': 'train', # or 'train' 
}
dataloader_args = {
    'batch_size': 128, # Adjust per your system capacity
    'num_workers': 10,
    'persistent_workers': True
}
sampler = 'SequentialFixedWindowSampler'
sampler_args = { 
                'window_length': 3.280,
                'drop_short': False,
                'min_window_length': 0.88
                }

dataset, data_loader, max_neural_input = get_brainset_data_loader(
    dataset_args=dataset_args,
    sampler = sampler,
    sampler_args = sampler_args,
    dataloader_args = dataloader_args,
    sampling_rate = 200,
    dataset_cfg = os.path.join(config_dir, 'pm_t.yaml'),
    data_root = data_root
)

sessions = dataset.get_session_ids() # list of sessions in your dataset
sampling_intervals = dataset.get_sampling_intervals() # list of sampling intervals for each session

In [10]:
max_neural_ids = []
for session, interval in sampling_intervals.items():
    for start, end in zip(interval.start, interval.end):
        sample = dataset.get_recording_data(session).slice(start,end)
        neural_input = sample.spikes.unit_index
        max_neural_ids.append(neural_input.max())
        

In [13]:
max_neural_ids

[np.int64(54),
 np.int64(54),
 np.int64(54),
 np.int64(54),
 np.int64(54),
 np.int64(54),
 np.int64(54),
 np.int64(54),
 np.int64(54),
 np.int64(54),
 np.int64(54),
 np.int64(54),
 np.int64(54),
 np.int64(54),
 np.int64(54),
 np.int64(54),
 np.int64(54),
 np.int64(54),
 np.int64(54),
 np.int64(54),
 np.int64(54),
 np.int64(54),
 np.int64(54),
 np.int64(54),
 np.int64(54),
 np.int64(54),
 np.int64(54),
 np.int64(53),
 np.int64(58),
 np.int64(58),
 np.int64(58),
 np.int64(58),
 np.int64(58),
 np.int64(58),
 np.int64(58),
 np.int64(58),
 np.int64(58),
 np.int64(58),
 np.int64(58),
 np.int64(58),
 np.int64(58),
 np.int64(58),
 np.int64(58),
 np.int64(58),
 np.int64(58),
 np.int64(58),
 np.int64(58),
 np.int64(58),
 np.int64(58),
 np.int64(58),
 np.int64(58),
 np.int64(58),
 np.int64(58),
 np.int64(58),
 np.int64(58),
 np.int64(63),
 np.int64(63),
 np.int64(63),
 np.int64(44),
 np.int64(44),
 np.int64(44),
 np.int64(44),
 np.int64(44),
 np.int64(44),
 np.int64(44),
 np.int64(44),
 np.int64(

In [ ]:
import jax 
import jax.numpy as jnp 
import numpy as np
from tqdm import tqdm 

metrics = {}  # New: store metrics per group
all_preds = []
all_targets = []
all_dataset_group_idxs = []
max_neural_input = []

for batch_idx, batch in tqdm(enumerate(data_loader)):
    batch = {k: jax.device_put(np.array(v)) for k, v in batch.items()}
    dataset_group_idxs = batch["dataset_group_idx"]
    inputs = batch["neural_input"]
    targets = batch["behavior_input"]

    all_targets.append(targets)
    all_dataset_group_idxs.append(dataset_group_idxs)
    max_neural_input.append(inputs.shape[-2])
all_targets = jnp.concatenate(all_targets, axis=0)
all_dataset_group_idxs = jnp.concatenate(all_dataset_group_idxs, axis=0)

for group_idx in tqdm(jnp.unique(all_dataset_group_idxs)):
    group_targets = all_targets[all_dataset_group_idxs == group_idx]
    variance = jnp.var(group_targets.reshape(-1, 2), axis=0)
    metrics[int(group_idx)] = {
        "variance": variance
    }
metrics

0it [00:00, ?it/s]/tmp/ipykernel_890596/2353987421.py:12: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  batch = {k: jax.device_put(np.array(v)) for k, v in batch.items()}
19it [00:01, 12.13it/s]
100%|██████████| 2/2 [00:00<00:00,  3.01it/s]


{10: {'variance': Array([0.99999994, 0.99999994], dtype=float32)},
 11: {'variance': Array([1., 1.], dtype=float32)}}

In [3]:
for batch_idx, batch in tqdm(enumerate(data_loader)):
    batch = {k: jax.device_put(np.array(v)) for k, v in batch.items()}
    dataset_group_idxs = batch["dataset_group_idx"]
    inputs = batch["neural_input"]
    targets = batch["behavior_input"]
    break

NameError: name 'tqdm' is not defined